In [1]:
import os
import pandas as pd
import statistics

import IPython
import gradio
import glob

from sklearn.metrics import accuracy_score

In [2]:
audio_path = "/Users/nimamanaf/Library/CloudStorage/GoogleDrive-ndizbin14@ku.edu.tr/My Drive/Techno/technob/docs/examples/cse.WAV"


In [3]:
import torch
model = torch.hub.load('harritaylor/torchvggish', 'vggish')

Using cache found in /Users/nimamanaf/.cache/torch/hub/harritaylor_torchvggish_master


: 

: 

In [1]:
from qdrant_client import QdrantClient


In [4]:
client = QdrantClient("http://localhost:6333") # Connect to existing Qdrant instance, for production

In [5]:
from qdrant_client import models
client.recreate_collection(
  collection_name='startups',
  vectors_config=models.VectorParams(size=768, distance="Cosine")
)

True

In [9]:
class NeuralSearcher:

    def __init__(self, collection_name):
        self.collection_name = collection_name
        # Initialize encoder model
        self.model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens', device='cpu')
        # initialize Qdrant client
        self.qdrant_client = QdrantClient(host='localhost', port=6333)

    # The search function looks as simple as possible:

    def search(self, text: str):
        # Convert text query into vector
        vector = self.model.encode(text).tolist()

        # Use `vector` for search for closest vectors in the collection
        search_result = self.qdrant_client.search(
            collection_name=self.collection_name,
            query_vector=vector,
            query_filter=None,  # We don't want any filters for now
            top=5  # 5 the most closest results is enough
        )

        # `search_result` contains found vector ids with similarity scores along with the stored payload
        # In this function we are interested in payload only
        payloads = [hit.payload for hit in search_result]
        return payloads

In [14]:
# Create collection to store books
client.recreate_collection(
    collection_name="my_books",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

True

In [15]:
# Let's vectorize descriptions and upload to qdrant

qdrant.upload_records(
    collection_name="my_books",
    records=[
        models.Record(
            id=idx,
            vector=encoder.encode(doc["description"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(documents)
    ]
)

In [18]:
# Let's now search for something

hits = qdrant.search(
    collection_name="my_books",
    query_vector=encoder.encode("What are possbile ways to DJ ").tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'description': 'Zoom is a video conferencing software that helps users hold online meetings, training sessions, and webinars.', 'name': 'Zoom'} score: 0.12417556
{'description': 'Uber develops, markets, and operates a ride-sharing mobile application that allows consumers to submit a trip request.', 'name': 'Uber'} score: 0.11771207
{'description': 'Airbnb is an online community marketplace for people to list, discover, and book accommodations.', 'name': 'Airbnb'} score: 0.1114932


In [14]:
hits = qdrant.search(
    collection_name="my_books",
    query_vector=encoder.encode("Tyranic society").tolist(),
    query_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="year",
                range=models.Range(
                    gte=2000
                )
            )
        ]
    ),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)